# Compare Global Cities: New York City, Toronto, London, and Paris

New York City, London, Paris and Shanghai are all global cities. I have never been to New York City, London. However I'm quite familiar with Paris and Shanghai. I'm going to compare these four cities to find their similarities and dissimilarities. Expecting I would enjoy more if I had the chance travel to New York City and London.

In [1]:
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge BeautifulSoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.2.5-py37hefd8a0e_0 --> 4.4.1-py36h7ec2d77_0 conda-forge


lxml-4.4.1           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ cond

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

%matplotlib inline 

import json
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans 
from geopy.geocoders import Nominatim
import folium
from bs4 import BeautifulSoup

#### First, let's compare the four cities on an overall base.

I collect data from Wikipedia pages below:
1. New York City: https://en.wikipedia.org/wiki/New_York_City; https://en.wikipedia.org/wiki/Neighborhoods_in_New_York_City
2. Toronto: https://en.wikipedia.org/wiki/Toronto; https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto
3. London: https://en.wikipedia.org/wiki/London; https://en.wikipedia.org/wiki/List_of_London_boroughs; https://en.wikipedia.org/wiki/London_postal_district
4. Paris: https://en.wikipedia.org/wiki/Paris; https://en.wikipedia.org/wiki/Arrondissements_of_Paris

In [3]:
column_names=['City', 'Population (million)', 'Area (km2)', 'Sub-district numbers', 'Sub-sub-district numbers']#, 'Neighborhood numbers']
cities=pd.DataFrame(columns=column_names)
row_ny=['New York City', 20.0, 784, 5, 57]
row_tr=['Toronto', 2.7, 630, 9, 140]
row_ld=['London', 8.8, 1572, 33, None]
row_pr=['Paris', 2.1, 105, 20, None]
cities.loc[0]=row_ny
cities.loc[1]=row_tr
cities.loc[2]=row_ld
cities.loc[3]=row_pr
cities

,City,Population (million),Area (km2),Sub-district numbers,Sub-sub-district numbers
0,New York City,20.0,784,5,57
1,Toronto,2.7,630,9,140
2,London,8.8,1572,33,None
3,Paris,2.1,105,20,None


1. New York City has 5 *boroughs* and 57 *communities*. Each community has numbers of *neighborhoods*. But their names and borders are not officially defined, and they change from time to time.
2. Toronto has 140 officially recognized *neighbourhoods* in 9 *districts/boroughs*.
3. London area is the largest, it has 32 *boroughs* and City of London. So we can count as 33 boroughs.
4. Paris only have 20 *arrondissements*, somewhat like boroughs in New York City or London, but they are rather small. And it's hard to define smaller units under arrondissement.

But if we analyze nearby venue categories based on this range, it seems make little sense. The areas of smallest units of each city range broad.

#### If we compare the four cities' center parts.

In [4]:
column_names=['City', 'Borough / District', 'Area (km2)', 'Sub-district numbers']
city_centers=pd.DataFrame(columns=column_names)
center_ny=['New York City', 'Manhattan', 59, 40]
center_tr=['Toronto', 'Old Toronto', 97, 38]
center_ld=['London', 'Inner London', 319, 183]
center_pr=['Paris', 'Paris', 105, 22]
city_centers.loc[0]=center_ny
city_centers.loc[1]=center_tr
city_centers.loc[2]=center_ld
city_centers.loc[3]=center_pr
city_centers

,City,Borough / District,Area (km2),Sub-district numbers
0,New York City,Manhattan,59,40
1,Toronto,Old Toronto,97,38
2,London,Inner London,319,183
3,Paris,Paris,105,22


Considering Paris is too small, so take the whole Paris as the *center part*. Based on center parts and their sub-districts, venues analysis makes more sense.

### 1. Get sub-districts' infos

I have already got the four cities' sub-districts data. They were saved them as CSV files. If interested, please visit the other python notebooks named as the cities' names respectively. So here I will read those CSV files directly.

In [76]:
df_newyork=pd.read_csv("manhattan_neighborhoods.csv", index_col=0)
df_newyork['City']='New York City'
ny_col=df_newyork.columns
ny_col=[ny_col[-1]]+list(ny_col[:-1])
df_newyork=df_newyork[ny_col]

print('New York points: {}'.format(df_newyork.shape[0]))
df_newyork.head()

New York points: 40


,City,Borough,Neighborhood,Latitude,Longitude
0,New York City,Manhattan,Marble Hill,40.876551,-73.910660
1,New York City,Manhattan,Chinatown,40.715618,-73.994279
2,New York City,Manhattan,Washington Heights,40.851903,-73.936900
3,New York City,Manhattan,Inwood,40.867684,-73.921210
4,New York City,Manhattan,Hamilton Heights,40.823604,-73.949688


In [77]:
df_toronto=pd.read_csv("old_toronto_neighbourhoods.csv", index_col=0)
df_toronto['City']='Toronto'
tr_col=df_toronto.columns
tr_col=[tr_col[-1]]+list(tr_col[:-1])
df_toronto=df_toronto[tr_col]

print('Toronto points: {}'.format(df_toronto.shape[0]))
df_toronto.head()

Toronto points: 38


,City,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,Toronto,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,Toronto,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,Toronto,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,Toronto,M4M,East Toronto,Studio District,43.659526,-79.340923
44,Toronto,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [78]:
df_london=pd.read_csv("london_postcode_districts.csv", index_col=0)
df_london['City']='London'
ld_col=df_london.columns
ld_col=[ld_col[-1]]+list(ld_col[:-1])
df_london=df_london[ld_col]

print('London points: {}'.format(df_london.shape[0]))
df_london.head()

London points: 183


,City,Postcode area,Postcode area name,Postcode district,District name,Latitude,Longitude
0,London,E,Eastern,E1,Head district,51.517389,-0.059507
1,London,E,Eastern,E10,Leyton,51.567991,-0.014095
2,London,E,Eastern,E11,Leytonstone,51.568739,0.013628
3,London,E,Eastern,E12,Manor Park,51.550735,0.053017
4,London,E,Eastern,E13,Plaistow,51.527740,0.026491


In [79]:
df_paris=pd.read_csv("paris_arrondissements.csv", index_col=0)
df_paris['City']='Paris'
pr_col=df_paris.columns
pr_col=[pr_col[-1]]+list(pr_col[:-1])
df_paris=df_paris[pr_col]
#df_paris['Arrondissement']=df_paris['Arrondissement'].astype(str)

print('Paris points: {}'.format(df_paris.shape[0]))
df_paris

Paris points: 22


,City,Arrondissement,Name,Zip code,Latitude,Longitude,Area (km2)
0,Paris,1st,Louvre,75001,48.8592,2.3417,1.83
1,Paris,2nd,Bourse,75002,48.8655,2.3426,0.99
2,Paris,3rd,Temple,75003,48.8637,2.3615,1.17
3,Paris,4th,Hôtel-de-Ville,75004,48.8601,2.3507,1.60
4,Paris,5th,Panthéon,75005,48.8448,2.3471,2.54
5,Paris,6th,Luxembourg,75006,48.8493,2.3300,2.15
6,Paris,7th,Palais-Bourbon,75007,48.8565,2.3210,4.09
7,Paris,8th,Élysée,75008,48.8763,2.3183,3.88
8,Paris,9th,Opéra,75009,48.8718,2.3399,2.18
9,Paris,10th,Entrepôt,75010,48.8709,2.3561,2.89


In [80]:
cols=['City', 'District', 'Sub-district', 'Latitude', 'Longitude']
all_districts=pd.DataFrame(columns=cols)

df1=df_newyork[:]
df1.columns=cols

df2=df_toronto[['City', 'Borough', 'PostalCode', 'Latitude', 'Longitude']]
df2.columns=cols

df3=df_london[['City', 'Postcode area', 'Postcode district', 'Latitude', 'Longitude']]
df3.columns=cols

df4=df_paris[['City', 'Arrondissement', 'Name', 'Latitude', 'Longitude']]
df4.columns=cols

all_districts=pd.concat([df1,df2,df3,df4])
all_districts.head()

,City,District,Sub-district,Latitude,Longitude
0,New York City,Manhattan,Marble Hill,40.876551,-73.910660
1,New York City,Manhattan,Chinatown,40.715618,-73.994279
2,New York City,Manhattan,Washington Heights,40.851903,-73.936900
3,New York City,Manhattan,Inwood,40.867684,-73.921210
4,New York City,Manhattan,Hamilton Heights,40.823604,-73.949688


In [81]:
all_districts.shape

(283, 5)

In [82]:
all_districts.to_csv('all_district.csv')

### 2. Get Nearby Venues

In [83]:
CLIENT_ID = 'UT4U4XLCPJ5C5BJ20ABBEINT5HRIZNLWY3FMRXAZUN4OYE5L'
CLIENT_SECRET = 'NX3JTOYEVWPQ1JPIOOC3I1FZMCF1BQNKKFWNRULBY1V2NIA4'
VERSION = '20190801'

In [84]:
# define a function to return each coordinate's nearby venues
def getNearbyVenues(row, json_results, all_venues):
    items=json_results['response']['groups'][0]['items']
    for item in items:
        venue=item['venue']
        v_name=venue['name']
        v_lat=venue['location']['lat']
        v_lon=venue['location']['lng']
        v_cat=venue['categories'][0]['name']
        all_venues=all_venues.append({'City': row['City'], 
                                      'District': row['District'], 
                                      'Sub-district': row['Sub-district'], 
                                      'Sub-district Latitude': row['Latitude'],
                                      'Sub-district Longitude': row['Longitude'],
                                      'Venue': v_name, 
                                      'Venue Latitude': v_lat, 
                                      'Venue Longitude': v_lon, 
                                      'Venue Category': v_cat}, 
                                     ignore_index=True)
    return(all_venues)

In [85]:
# get every nearby venues according to postal codes' coordinates
# this process might take several minutes
col_venues=['City', 'District', 'Sub-district', 'Sub-district Latitude', 'Sub-district Longitude', 
               'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
all_venues=pd.DataFrame(columns=col_venues)

LIMIT=100
radius=500

for index, row in all_districts.iterrows():
    url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        row['Latitude'],
        row['Longitude'],
        radius, 
        LIMIT)
    results = requests.get(url).json()
    all_venues=getNearbyVenues(row, results, all_venues)
all_venues.shape

(16127, 9)

In [86]:
all_venues.head()

,City,District,Sub-district,Sub-district Latitude,Sub-district Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York City,Manhattan,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,New York City,Manhattan,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,New York City,Manhattan,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,New York City,Manhattan,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,New York City,Manhattan,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [87]:
all_venues.to_csv('all_venues.csv')

In [88]:
df_dummies=all_venues[['Sub-district', 'Venue Category']]
all_venues_onehot=pd.get_dummies(df_dummies, columns=['Venue Category'], prefix="", prefix_sep="")
all_venues_onehot.head()

,Sub-district,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alsatian Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Ch'ti Restaurant,Champagne Bar,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Gym,College Quad,College Rec Center,College Residence Hall,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Convention Center,Cooking School,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cultural Center,Cupcake Shop,Currywurst Joint,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,M

In [89]:
all_venues_onehot.shape

(16127, 468)

In [90]:
all_venues_groupby=all_venues_onehot.groupby('Sub-district').mean().reset_index()
all_venues_groupby.head()

,Sub-district,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alsatian Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Basque Restaurant,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Ch'ti Restaurant,Champagne Bar,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Gym,College Quad,College Rec Center,College Residence Hall,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Convention Center,Cooking School,Corsican Restaurant,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cultural Center,Cupcake Shop,Currywurst Joint,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Iraqi Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,M

In [91]:
all_venues_groupby.shape

(283, 468)

In [92]:
all_venues_groupby.to_csv('all_venues_groupby.csv')

In [93]:
col_top10=['Sub-district', 
           '1st popular catogory',
           '2nd popular catogory',
           '3rd popular catogory',
           '4th popular catogory', 
           '5th popular catogory', 
           '6th popular catogory', 
           '7th popular catogory', 
           '8th popular catogory', 
           '9th popular catogory', 
           '10th popular catogory']
all_venues_top10_category=pd.DataFrame(columns=col_top10)
all_venues_top10_category['Sub-district']=all_venues_groupby['Sub-district']
len(all_venues_top10_category.index.values)

283

In [94]:
def get_top10 (row):
    row_t=row.iloc[1:]
    row_t_sorted=row_t.sort_values(ascending=False)
    row_t_top10=row_t_sorted[0:10]
    top10=row_t_top10.index.values

    return top10

In [95]:
for index, row in all_venues_groupby.iterrows():
    top10=get_top10(row)
    all_venues_top10_category.iloc[index,1:]=top10  
all_venues_top10_category.head()

,Sub-district,1st popular catogory,2nd popular catogory,3rd popular catogory,4th popular catogory,5th popular catogory,6th popular catogory,7th popular catogory,8th popular catogory,9th popular catogory,10th popular catogory
0,Batignolles-Monceau,French Restaurant,Bar,Wine Bar,Hotel,Restaurant,Italian Restaurant,Thai Restaurant,Vietnamese Restaurant,Bakery,BBQ Joint
1,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Wine Shop,Gym,Italian Restaurant,Clothing Store,Ice Cream Shop,Sushi Restaurant
2,Bois-de-Boulogne,Park,Forest,French Restaurant,Garden,Yoga Studio,Falafel Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
3,Bois-de-Vincenne,Music Venue,Forest,Farmers Market,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service
4,Bourse,French Restaurant,Bistro,Cocktail Bar,Wine Bar,Hotel,Italian Restaurant,Café,Pizza Place,Boutique,Burger Joint


In [96]:
all_venues_top10_category.shape

(283, 11)

In [97]:
all_venues_top10_category.to_csv('all_venues_top10_category.csv')

### 3. Cluster Sub-districts

In [98]:
k=6
districts_cluster=all_venues_groupby.drop('Sub-district', 1)
districts_cluster.head()
kmeans = KMeans(n_clusters=k, random_state=0).fit(districts_cluster)
kmeans.labels_[0:10]

array([5, 5, 2, 5, 5, 5, 5, 5, 5, 5], dtype=int32)

In [99]:
# add clustering labels
all_venues_top10_category.insert(0, 'Cluster Label', kmeans.labels_)
all_districts_merged = all_districts[:]
all_districts_merged = all_districts_merged.join(all_venues_top10_category.set_index('Sub-district'), on='Sub-district')
all_districts_merged.head() 

,City,District,Sub-district,Latitude,Longitude,Cluster Label,1st popular catogory,2nd popular catogory,3rd popular catogory,4th popular catogory,5th popular catogory,6th popular catogory,7th popular catogory,8th popular catogory,9th popular catogory,10th popular catogory
0,New York City,Manhattan,Marble Hill,40.876551,-73.910660,5,Sandwich Place,Coffee Shop,Discount Store,Yoga Studio,Supplement Shop,Steakhouse,Spa,Shopping Mall,Seafood Restaurant,Clothing Store
1,New York City,Manhattan,Chinatown,40.715618,-73.994279,5,Chinese Restaurant,Cocktail Bar,Salon / Barbershop,Vietnamese Restaurant,American Restaurant,Bakery,Dumpling Restaurant,Ice Cream Shop,Bubble Tea Shop,Spa
2,New York City,Manhattan,Washington Heights,40.851903,-73.936900,5,Café,Bakery,Mobile Phone Shop,Grocery Store,Chinese Restaurant,New American Restaurant,Coffee Shop,Latin American Restaurant,Deli / Bodega,Italian Restaurant
3,New York City,Manhattan,Inwood,40.867684,-73.921210,5,Café,Mexican Restaurant,Lounge,Pizza Place,Pharmacy,Park,Spanish Restaurant,Wine Bar,Chinese Restaurant,Frozen Yogurt Shop
4,New York City,Manhattan,Hamilton Heights,40.823604,-73.949688,5,Pizza Place,Mexican Restaurant,Café,Coffee Shop,Yoga Studio,Chinese Restaurant,Liquor Store,Sushi Restaurant,Bakery,School


In [100]:
all_districts_merged.shape

(283, 16)

#### Show in the map

*Need to zoom manually to each city to see clusters.*

In [101]:
address = 'North Atlantic Ocean'

geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [102]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=3)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_districts_merged['Latitude'], 
                                  all_districts_merged['Longitude'], 
                                  all_districts_merged['Sub-district'], 
                                  all_districts_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 4. Examine Clusters

#### Cluster 1 --- *red points*

These points surround with lots of pubs/bars, cafes/coffee shops and restaurants, might be very cozy places to visit. Most in London, one in Toronto. Maybe living in Toronto postcode M4E area would be somewhat like living in those London parts.

In [103]:
all_districts_merged.loc[all_districts_merged['Cluster Label'] == 0, 
                         all_districts_merged.columns[list(range(0, 3)) + 
                                                      list(range(6, all_districts_merged.shape[1]))]]

,City,District,Sub-district,1st popular catogory,2nd popular catogory,3rd popular catogory,4th popular catogory,5th popular catogory,6th popular catogory,7th popular catogory,8th popular catogory,9th popular catogory,10th popular catogory
37,Toronto,East Toronto,M4E,Trail,Pub,Health Food Store,Other Great Outdoors,Neighborhood,Duty-free Shop,Eastern European Restaurant,Electronics Store,Fabric Shop,Empanada Restaurant
3,London,E,E12,Gym / Fitness Center,Restaurant,Turkish Restaurant,Gas Station,Pub,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Fabric Shop,Empanada Restaurant
4,London,E,E13,Café,Pub,Bus Station,Gym,Flea Market,Fishing Store,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store
13,London,E,E3,Pub,Bus Stop,Grocery Store,Hotel,Locksmith,Café,Light Rail Station,Bar,Burger Joint,Convenience Store
17,London,E,E7,Grocery Store,Pub,Café,Fast Food Restaurant,Bus Stop,Train Station,Comfort Food Restaurant,Chinese Restaurant,Fish & Chips Shop,Moving Target
19,London,E,E9,Pub,Coffee Shop,Café,Yoga Studio,Train Station,Grocery Store,Food & Drink Shop,Park,Pizza Place,Bus Stop
54,London,N,N15,Arepa Restaurant,Hardware Store,South American Restaurant,Bus Stop,Fried Chicken Joint,Pub,Café,Electronics Store,Empanada Restaurant,English Restaurant
55,London,N,N16,Pub,Café,Restaurant,Pizza Place,Italian Restaurant,Gift Shop,Cocktail Bar,Bakery,Coffee Shop,Grocery Store
56,London,N,N17,Pub,Gym / Fitness Center,Convenience Store,Discount Store,Café,Supermarket,Gas Station,Park,Greek Restaurant,Bar
67,London,N,N5,Café,Pub,Deli / Bodega,Vietnamese Restaurant,Bar,Thai Restaurant,Cheese Shop,Tapas Restaurant,Coffee Shop,Middle Eastern Restaurant


#### Cluster 2 --- *purple points*

One in New York Manhattan, several in Old Toronto, and lots in London. Very close to center downtown. Surrounded with lots of pubs and cafes, having numbers of restaurants, most of which are fast food and foreign cuisines. 

In [104]:
all_districts_merged.loc[all_districts_merged['Cluster Label'] == 1, 
                         all_districts_merged.columns[list(range(0, 3)) + 
                                                      list(range(6, all_districts_merged.shape[1]))]]

,City,District,Sub-district,1st popular catogory,2nd popular catogory,3rd popular catogory,4th popular catogory,5th popular catogory,6th popular catogory,7th popular catogory,8th popular catogory,9th popular catogory,10th popular catogory
26,New York City,Manhattan,Morningside Heights,Bookstore,Coffee Shop,Park,American Restaurant,Deli / Bodega,Food Truck,Sandwich Place,Burger Joint,Tennis Court,Frozen Yogurt Shop
49,Toronto,Central Toronto,M4V,Pub,Coffee Shop,Vietnamese Restaurant,Sports Bar,Fried Chicken Joint,Supermarket,Sushi Restaurant,Liquor Store,Bagel Shop,Restaurant
51,Toronto,Downtown Toronto,M4X,Restaurant,Coffee Shop,Café,Pizza Place,Bakery,Pub,Flower Shop,Italian Restaurant,Butcher,Park
53,Toronto,Downtown Toronto,M5A,Coffee Shop,Café,Park,Bakery,Breakfast Spot,Pub,Mexican Restaurant,Bank,French Restaurant,Event Space
65,Toronto,Central Toronto,M5R,Coffee Shop,Sandwich Place,Café,Pizza Place,Indian Restaurant,History Museum,Pharmacy,Park,Burger Joint,Cosmetics Shop
75,Toronto,Downtown Toronto,M6G,Grocery Store,Café,Park,Restaurant,Baby Store,Diner,Athletics & Sports,Coffee Shop,Convenience Store,Nightclub
78,Toronto,West Toronto,M6K,Breakfast Spot,Café,Coffee Shop,Yoga Studio,Pet Store,Burrito Place,Caribbean Restaurant,Climbing Gym,Restaurant,Performing Arts Venue
84,Toronto,West Toronto,M6S,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Gym,French Restaurant,Dessert Shop,Diner,Latin American Restaurant
0,London,E,E1,Sandwich Place,Coffee Shop,Indian Restaurant,Hotel,Grocery Store,Pub,Burger Joint,Turkish Restaurant,Fast Food Restaurant,Ice Cream Shop
2,London,E,E11,Pub,Café,Coffee Shop,Fast Food Restaurant,Platform,Grocery Store,Bakery,Pharmacy,Supermarket,Mediterranean Restaurant


#### Cluster 3 --- *blue points*

These are park areas. Out of parks, you can visit a lot of nice restaurants in styles of Japanese, Eastern European, English, or Ethiopian.

In [105]:
all_districts_merged.loc[all_districts_merged['Cluster Label'] == 2, 
                         all_districts_merged.columns[list(range(0, 3)) + 
                                                      list(range(6, all_districts_merged.shape[1]))]]

,City,District,Sub-district,1st popular catogory,2nd popular catogory,3rd popular catogory,4th popular catogory,5th popular catogory,6th popular catogory,7th popular catogory,8th popular catogory,9th popular catogory,10th popular catogory
37,New York City,Manhattan,Stuyvesant Town,Park,Bar,Playground,Cocktail Bar,Heliport,Basketball Court,Gas Station,Harbor / Marina,Pet Service,Boat or Ferry
44,Toronto,Central Toronto,M4N,Park,Bus Line,Swim School,Photography Studio,Yoga Studio,Falafel Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
45,Toronto,Central Toronto,M4P,Gym,Gym / Fitness Center,Asian Restaurant,Clothing Store,Sandwich Place,Hotel,Park,Grocery Store,Breakfast Spot,Food & Drink Shop
48,Toronto,Central Toronto,M4T,Trail,Summer Camp,Playground,Yoga Studio,Falafel Restaurant,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
50,Toronto,Downtown Toronto,M4W,Park,Trail,Building,Playground,Yoga Studio,Falafel Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant
63,Toronto,Central Toronto,M5N,Music Venue,Garden,Home Service,Farm,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant
64,Toronto,Central Toronto,M5P,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Fabric Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store
76,Toronto,West Toronto,M6H,Supermarket,Bakery,Pharmacy,Bank,Discount Store,Brewery,Café,Middle Eastern Restaurant,Park,Bar
87,Toronto,East Toronto,M7Y,Brewery,Fast Food Restaurant,Garden Center,Garden,Skate Park,Auto Workshop,Gym / Fitness Center,Pizza Place,Farmers Market,Smoke Shop
1,London,E,E10,Convenience Store,Park,Cricket Ground,Farm,Train Station,Coffee Shop,Hotel,Electronics Store,Eastern European Restaurant,Empanada Restaurant


#### Cluster 4 --- *cyan blue points*

In [106]:
all_districts_merged.loc[all_districts_merged['Cluster Label'] == 3, 
                         all_districts_merged.columns[list(range(0, 3)) + 
                                                      list(range(6, all_districts_merged.shape[1]))]]

,City,District,Sub-district,1st popular catogory,2nd popular catogory,3rd popular catogory,4th popular catogory,5th popular catogory,6th popular catogory,7th popular catogory,8th popular catogory,9th popular catogory,10th popular catogory
108,London,SE,SE3,Furniture / Home Store,Yoga Studio,Farm,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant


#### Cluster 5 --- *green points*

In [107]:
all_districts_merged.loc[all_districts_merged['Cluster Label'] == 4, 
                         all_districts_merged.columns[list(range(0, 3)) + 
                                                      list(range(6, all_districts_merged.shape[1]))]]

,City,District,Sub-district,1st popular catogory,2nd popular catogory,3rd popular catogory,4th popular catogory,5th popular catogory,6th popular catogory,7th popular catogory,8th popular catogory,9th popular catogory,10th popular catogory
83,London,NW,NW7,Park,Pharmacy,Yoga Studio,Fabric Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant


#### Cluster 6 --- *orange points*

Apparently these are city center parts. Almost all parts of Manhattan and Paris are belong to this genre. 

In [108]:
all_districts_merged.loc[all_districts_merged['Cluster Label'] == 5, 
                         all_districts_merged.columns[list(range(0, 3)) + 
                                                      list(range(6, all_districts_merged.shape[1]))]]

,City,District,Sub-district,1st popular catogory,2nd popular catogory,3rd popular catogory,4th popular catogory,5th popular catogory,6th popular catogory,7th popular catogory,8th popular catogory,9th popular catogory,10th popular catogory
0,New York City,Manhattan,Marble Hill,Sandwich Place,Coffee Shop,Discount Store,Yoga Studio,Supplement Shop,Steakhouse,Spa,Shopping Mall,Seafood Restaurant,Clothing Store
1,New York City,Manhattan,Chinatown,Chinese Restaurant,Cocktail Bar,Salon / Barbershop,Vietnamese Restaurant,American Restaurant,Bakery,Dumpling Restaurant,Ice Cream Shop,Bubble Tea Shop,Spa
2,New York City,Manhattan,Washington Heights,Café,Bakery,Mobile Phone Shop,Grocery Store,Chinese Restaurant,New American Restaurant,Coffee Shop,Latin American Restaurant,Deli / Bodega,Italian Restaurant
3,New York City,Manhattan,Inwood,Café,Mexican Restaurant,Lounge,Pizza Place,Pharmacy,Park,Spanish Restaurant,Wine Bar,Chinese Restaurant,Frozen Yogurt Shop
4,New York City,Manhattan,Hamilton Heights,Pizza Place,Mexican Restaurant,Café,Coffee Shop,Yoga Studio,Chinese Restaurant,Liquor Store,Sushi Restaurant,Bakery,School
5,New York City,Manhattan,Manhattanville,Mexican Restaurant,Park,Seafood Restaurant,Italian Restaurant,Coffee Shop,Café,BBQ Joint,Bank,Bar,Grocery Store
6,New York City,Manhattan,Central Harlem,African Restaurant,Chinese Restaurant,Seafood Restaurant,Cosmetics Shop,Art Gallery,Public Art,French Restaurant,American Restaurant,Bar,Gym
7,New York City,Manhattan,East Harlem,Mexican Restaurant,Deli / Bodega,Bakery,Thai Restaurant,Latin American Restaurant,Steakhouse,Grocery Store,Performing Arts Venue,Clothing Store,Sandwich Place
8,New York City,Manhattan,Upper East Side,Exhibit,Italian Restaurant,Art Gallery,Coffee Shop,Bakery,Gym / Fitness Center,Spa,Juice Bar,Cocktail Bar,French Restaurant
9,New York City,Manhattan,Yorkville,Italian Restaurant,Coffee Shop,Gym,Bar,Pizza Place,Sushi Restaurant,Wine Shop,Deli / Bodega,Japanese Restaurant,Mexican Restaurant
